# FastGRNN in Keras

This is modified to Keras implementation of FastGRNN from a original simple notebook that illustrates the usage of tensorflow implementation. This is just only training result. The USPS dataset is used as with the original code.  
Original code is as follows.  
https://github.com/microsoft/EdgeML/tree/master/examples/tf/FastCells
  
Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.  

The original code license is as follows.  
Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT license.

In [1]:
import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os
from tensorflow.keras.layers import Dense, Input, RNN
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from rnn import FastGRNNCellKeras

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# USPS Data

--- Here is completely copied from the original code. ---  
  
  
It is assumed that the USPS data has already been downloaded and processed with [fetch_usps.py](fetch_usps.py) and [process_usps.py](process_usps.py), and is present in the `./usps10` subdirectory.

Note: Even though usps10 is not a time-series dataset, it can be assumed as, a time-series where each row is coming in at one single time.
So the number of timesteps = 16 and inputDims = 16

In [2]:
#Loading and Pre-processing dataset for FastCells
dataDir = "usps10"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  256
Num classes:  10


# Model Parameters

---Here is completely copied from the original code.---  
  

FastRNN and FastGRNN work for most of the hyper-parameters with which you could acheive decent accuracies on LSTM/GRU. Over and above that, you can use low-rank, sparsity and quatization to reduce model size upto 45x when compared to LSTM/GRU.

In [3]:
cell = "FastGRNN" # Choose between FastGRNN, FastRNN, UGRNN, GRU and LSTM

inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 300
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

# FastCell Graph Object

In [4]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCellKeras(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
    
###!!!Other Cells is not implemented for Keras!!!!###
elif cell == "FastRNN":
    FastCell = FastRNNCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "UGRNN":
    FastCell = UGRNNLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "GRU":
    FastCell = GRULRCell(hiddenDims, update_non_linearity=update_non_linearity,
                         wRank=wRank, uRank=uRank)
elif cell == "LSTM":
    FastCell = LSTMLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                          wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

# FastCell Trainer Model for Keras

In [7]:
x = inputs = Input(shape=[int(dataDimension / inputDims), inputDims], name='input')
layer_shape = K.int_shape(x)#x.get_shape()
print(layer_shape)
x = RNN(FastCell, return_sequences=False, name='rnn')(x)
out = Dense(numClasses, activation='softmax', name='dense')(x)
model = Model(inputs=inputs, outputs=out)
Adam_ = Adam(lr=0.01)
model.compile(optimizer=Adam_, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
#Y = tf.placeholder("float", [None, numClasses])

(None, 16, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 16, 16)            0         
_________________________________________________________________
rnn (RNN)                    (None, 32)                3204      
_________________________________________________________________
dense (Dense)                (None, 10)                330       
Total params: 1,932
Trainable params: 1,932
Non-trainable params: 0
_________________________________________________________________


# Training

In [8]:
Xtrain_ = np.reshape(Xtrain, [-1, int(dataDimension / inputDims), inputDims])
Xtest_ = np.reshape(Xtest, [-1, int(dataDimension / inputDims), inputDims])
model.fit(Xtrain_, Ytrain, batchSize, epochs=totalEpochs, validation_data=(Xtest_, Ytest))

Train on 7291 samples, validate on 2007 samples
Epoch 1/300
7291/7291 [==============================] - 1s 167us/sample - loss: 1.3699 - acc: 0.5186 - val_loss: 1.0842 - val_acc: 0.6413
Epoch 2/300
7291/7291 [==============================] - 1s 105us/sample - loss: 0.8093 - acc: 0.7254 - val_loss: 0.8386 - val_acc: 0.7235
Epoch 3/300
7291/7291 [==============================] - 1s 98us/sample - loss: 0.6489 - acc: 0.7855 - val_loss: 0.7413 - val_acc: 0.7534
Epoch 4/300
7291/7291 [==============================] - 1s 92us/sample - loss: 0.5442 - acc: 0.8238 - val_loss: 0.6908 - val_acc: 0.7778
Epoch 5/300
7291/7291 [==============================] - 1s 91us/sample - loss: 0.4598 - acc: 0.8526 - val_loss: 0.6257 - val_acc: 0.8037
Epoch 6/300
7291/7291 [==============================] - 1s 91us/sample - loss: 0.3864 - acc: 0.8745 - val_loss: 0.5689 - val_acc: 0.8286
Epoch 7/300
7291/7291 [==============================] - 1s 96us/sample - loss: 0.3374 - acc: 0.8940 - val_loss: 0.5384 - 

7291/7291 [==============================] - 1s 83us/sample - loss: 0.0292 - acc: 0.9919 - val_loss: 0.4543 - val_acc: 0.9178
Epoch 118/300
7291/7291 [==============================] - 1s 84us/sample - loss: 0.0224 - acc: 0.9945 - val_loss: 0.4663 - val_acc: 0.9143
Epoch 119/300
7291/7291 [==============================] - 1s 85us/sample - loss: 0.0467 - acc: 0.9852 - val_loss: 0.4694 - val_acc: 0.9048
Epoch 120/300
7291/7291 [==============================] - 1s 85us/sample - loss: 0.0542 - acc: 0.9834 - val_loss: 0.4369 - val_acc: 0.9118
Epoch 121/300
7291/7291 [==============================] - 1s 87us/sample - loss: 0.0477 - acc: 0.9848 - val_loss: 0.3828 - val_acc: 0.9188
Epoch 122/300
7291/7291 [==============================] - 1s 85us/sample - loss: 0.0528 - acc: 0.9827 - val_loss: 0.3884 - val_acc: 0.9173
Epoch 123/300
7291/7291 [==============================] - 1s 87us/sample - loss: 0.0490 - acc: 0.9823 - val_loss: 0.4114 - val_acc: 0.9163
Epoch 124/300
7291/7291 [=========

7291/7291 [==============================] - 1s 146us/sample - loss: 0.0113 - acc: 0.9977 - val_loss: 0.5045 - val_acc: 0.9188
Epoch 234/300
7291/7291 [==============================] - 1s 171us/sample - loss: 0.0065 - acc: 0.9990 - val_loss: 0.4588 - val_acc: 0.9243
Epoch 235/300
7291/7291 [==============================] - 1s 150us/sample - loss: 0.0077 - acc: 0.9981 - val_loss: 0.5029 - val_acc: 0.9173
Epoch 236/300
7291/7291 [==============================] - 1s 125us/sample - loss: 0.0247 - acc: 0.9937 - val_loss: 0.5188 - val_acc: 0.9118
Epoch 237/300
7291/7291 [==============================] - 1s 115us/sample - loss: 0.0426 - acc: 0.9872 - val_loss: 0.5386 - val_acc: 0.9133
Epoch 238/300
7291/7291 [==============================] - 1s 99us/sample - loss: 0.0453 - acc: 0.9864 - val_loss: 0.4845 - val_acc: 0.9143
Epoch 239/300
7291/7291 [==============================] - 1s 115us/sample - loss: 0.0535 - acc: 0.9834 - val_loss: 0.4853 - val_acc: 0.9168
Epoch 240/300
7291/7291 [===